In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from lib.DrugUtil import flatten, DrugUtil
import pickle

import plotly.express as px
from lib.FhirDataUtil import FhirDataUtil
from lib.CCSDataUtil import CCSDataUtil

def printmd(string):
    display(string)

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_colwidth', None)

_fhir_conditions_file = "input/fhir_conditions_20221003.csv"
_fhir_medications_file = "input/fhir_medications_20221003.csv"
_fhir_medications_ref_file = "input/fhir_medications_references_20221003.csv"
_fhir_demographics_file = "input/fhir_demographics_20221113.csv"
_output_dir = 'output/'

_rxnorm_file = "input_static/RxTerms202203/RxTerms202203.txt"
_rxnorm_ingredients_file = "input_static/RxTerms202203/RxTermsIngredients202203.txt"
_rxclass_file = "input_static/_rxclass_2022-04-10.csv"

_ccs_demographics_file = "input/ccs_demographics.csv"
_ccs_medications_file = "input/ccs_medications.csv"
_ccs_conditions_file = "input/ccs_conditions.csv"
_ccs_medications_preprocessed_file = "input/ccs_medications_preprocessed.csv"

_pc_demographics_file = "input/pcornet/demographic_fhir.csv"
_pc_medications_file = "input/pcornet/med_pres_fhir_v2.csv"
_pc_medications_query_file = "input/pc_medications_query.xlsx"

# Chart Review
_cr_file = 'input/chart_review/Chart_Review1.2b.xlsx'

_cache_dir = 'cache/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
fhirUtil = FhirDataUtil()
drugUtil = DrugUtil()
ccsUtil = CCSDataUtil(drugUtil)


drugUtil.load(rxnorm_file=_rxnorm_file,
                        rxnorm_ingredients_file=_rxnorm_ingredients_file, 
                        rxclass_file=_rxclass_file)

fhirUtil.load_conditions(fhir_conditions_file=_fhir_conditions_file, remove_health_concerns=True)
fhirUtil.load_medications(medications_fhir_file=_fhir_medications_file, medications_ref_fhir_file=_fhir_medications_ref_file)
fhirUtil.load_demographics(fhir_demographics_file=_fhir_demographics_file)


ccsUtil.load_conditions(ccs_conditions_file= _ccs_conditions_file)
ccsUtil.load_medications(ccs_medications_file=_ccs_medications_file)
# ccsUtil.medications = ccsUtil.medications.drop_duplicates(subset=['user_id', 'rxcui'])
# ccsUtil.add_ingredient_column()
ccsUtil.load_demographics(_ccs_demographics_file)

fhir_meds = fhirUtil.medications
pc_demo = pd.read_csv(_pc_demographics_file)

Reading rxclass file...
Reading rxnorm file...
RAW FHIR conditions: 52621
FHIR conditions after pre-processing: 37248
----- Loading Medications ------
Raw FHIR records: 15061
After de-duplicated: 7403
Merge with reference counts:
left_only     5133
both          2236
right_only       0
Name: ref_merge, dtype: int64
Loaded FHIR medication records 31976  Includes multiple code mappings
Unique users with fhir medications:  1122
------  Done Loading Medications -----
Loaded CCS conditions records: 94735


/app/lib/CCSDataUtil.py:13: DtypeWarning: Columns (5,6,11,12,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  self.conditions = pd.read_csv(ccs_conditions_file, delimiter=',')
/app/lib/CCSDataUtil.py:20: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  self.medications_raw = pd.read_csv(ccs_medications_file)


Loaded demographics file with entries:  98877


/app/lib/CCSDataUtil.py:75: DtypeWarning: Columns (11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  self.demographics = pd.read_csv(ccs_demographics)


In [47]:
m = fhirUtil.medications.groupby(['provider', 'ref_merge'])['ref_merge'].count().reset_index(name="counts")
m['ref_merge'] = m['ref_merge'].replace({'left_only': 'no_med_ref', 'both': 'yes_med_ref'})
m = m[m['ref_merge'] != 'right_only']
c = m.set_index(['provider','ref_merge']).unstack(['ref_merge'])
c = c['counts']
c['total'] = c['no_med_ref'] + c['yes_med_ref']
c['percent_referenced'] = round(c['yes_med_ref']/(c['yes_med_ref']+c['no_med_ref']), 2)*100
c.sort_values('total', ascending=False)
c.to_csv('output/medication_references_by_provider.csv')
c

ref_merge,no_med_ref,yes_med_ref,total,percent_referenced
provider,,,,
openepic_shared_advantagecare_physicians,1,6,7,86.0
openepic_shared_adventist_health_west,6,0,6,0.0
openepic_shared_alameda_health_system,2,94,96,98.0
openepic_shared_allegheny_health_network,6,54,60,90.0
openepic_shared_allina_health_system,12,61,73,84.0
openepic_shared_ann_robert_h_lurie_children_s_hospital_of_chicago,6,0,6,0.0
openepic_shared_ardent,6,0,6,0.0
openepic_shared_arizona_community_physicians,8,0,8,0.0
openepic_shared_arkansas_children_s,9,0,9,0.0


In [44]:
fhir_meds = fhirUtil.medications_raw
fhir_meds = fhir_meds.drop_duplicates(subset='id')
fhir_meds_ref = fhirUtil.medications_ref
fhir_meds['med_ref_id'] = fhir_meds['medication_reference'].str[11:]
fhir_meds = fhir_meds[fhir_meds['dose_text'] == 'ordered']
fhir_meds = fhir_meds.merge(fhir_meds_ref[['id', 'coding_code']], left_on='med_ref_id', right_on='id', how="left", indicator='ref_merge')
fhir_meds = fhir_meds.rename(columns={'id_x': 'medication_id', 'id_y': 'reference_id'})
m = fhirUtil.medications.groupby(['provider', 'ref_merge'])['ref_merge'].count().reset_index(name="counts")
m

/tmp/ipykernel_474/3614698513.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fhir_meds['med_ref_id'] = fhir_meds['medication_reference'].str[11:]


,provider,ref_merge,counts
0,openepic_shared_advantagecare_physicians,left_only,1
1,openepic_shared_advantagecare_physicians,right_only,0
2,openepic_shared_advantagecare_physicians,both,6
3,openepic_shared_adventist_health_west,left_only,6
4,openepic_shared_adventist_health_west,right_only,0
5,openepic_shared_adventist_health_west,both,0
6,openepic_shared_alameda_health_system,left_only,2
7,openepic_shared_alameda_health_system,right_only,0
8,openepic_shared_alameda_health_system,both,94
9,openepic_shared_allegheny_health_network,left_only,6


In [50]:
fhir_meds[fhir_meds['provider'].str.contains('sutter')]

,user_id,uploaded_date,provider,medication_id,status,intent,medication_reference,text,authoredon,requester_display,recorder_display,course_of_therapy,dosage_text,dosage_patientinstruction,dosage_as_needed,dosage_route,dosage_method,dose_text,dose_quantity_value,dose_quantity_unit,start,repeats,quantity,quantity_unit,supply,supply_unit,med_ref_id,reference_id,coding_code,ref_merge
24,502639,2022-08-11,openepic_shared_sutter_health,eSTb-2WsuPKUR2Al4ibSYCsAsiLTVzQ9qQ9x72ab-jk83,active,order,Medication/eIk75Fg7HFG.BtpnFarGwDlvEUYxIW0-I4nS.MLz.moWens4Wl238m7eQBaL-sTBxYNdnxehEqPelUFhYbIw1fp6MNPVfjjEcEErppy7SesE3,gabapentin 300mg Cap,2021-02-16T21:28:33Z,NaN,Marisa M,Short course (acute) therapy,Take 300 mg by mouth three times daily \r\n,Take 300 mg by mouth three times daily \r\n,f,Oral,Take,ordered,300.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eIk75Fg7HFG.BtpnFarGwDlvEUYxIW0-I4nS.MLz.moWens4Wl238m7eQBaL-sTBxYNdnxehEqPelUFhYbIw1fp6MNPVfjjEcEErppy7SesE3,NaN,NaN,left_only
25,502639,2022-08-11,openepic_shared_sutter_health,erDuUFB3ygQRUlKVaXiNSZRU3gLa1BiFTvfAvSvbX.JI3,active,order,Medication/eSPLIsEMNnZxB4wlKHegy72PE5uCSXHv-7eBJSNQoMVwWe7HdYPbivb2rZti0H5BwKrHNHFxMiVI49hTZtR4yQ2DPrtXeY4FeoyxBg7YCh303,buPROPion 24Hr-XL 300mg Tab,2021-11-23T23:24:07Z,NaN,Marisa M,Short course (acute) therapy,Take 300 mg by mouth daily,Take 300 mg by mouth daily,f,Oral,Take,ordered,300.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eSPLIsEMNnZxB4wlKHegy72PE5uCSXHv-7eBJSNQoMVwWe7HdYPbivb2rZti0H5BwKrHNHFxMiVI49hTZtR4yQ2DPrtXeY4FeoyxBg7YCh303,NaN,NaN,left_only
26,502639,2022-08-11,openepic_shared_sutter_health,etQ3bKCIAzAy-3ed5GpwrwUPKV1xKUuC1KbnvTN7u.Fk3,active,order,Medication/eXnh6MUqEia72EyCx7r9OrUw0EzDLSTXInYPqpy7WELmRS821ATSnCZMfHekiQEUO2E8.S.sgE8AUD9io58goG54ssGbHERFu3b0oAixO.Lo3,Ubrelvy 100 MG Tabs,2022-06-09,"Steven W Nakano, MD","Steven W Nakano, MD",Short course (acute) therapy,Take 100 mg by mouth as directed as needed for Headache (May repeat after 2 hours),Take 100 mg by mouth as directed as needed for Headache (May repeat after 2 hours),t,Oral,Take,ordered,100.0,mg,2022-06-09,11.0,10.0,Tablet,NaN,NaN,eXnh6MUqEia72EyCx7r9OrUw0EzDLSTXInYPqpy7WELmRS821ATSnCZMfHekiQEUO2E8.S.sgE8AUD9io58goG54ssGbHERFu3b0oAixO.Lo3,NaN,NaN,left_only
221,456395,2022-09-01,openepic_shared_sutter_health,eCE87kBtKzR9hvRU9KvTSg6kEva3O.IjpnN2Y.W3fIZk3,active,order,Medication/epGSpBw4bXkctvVxMQKSRjlX-SWaJ5izz998FKEfDPGhb07mil2EFJELlvNcwUPjJbmSPqHhNf.Y8oHOdgIqMrxZGUKPjtqH4ns6HWG9soOU3,"cyanocobalamin 1,000mcg Tab",2012-04-05T01:07:08Z,NaN,"Maura Devereux, PAC",Short course (acute) therapy,"Take 1,000 mcg by mouth daily.","Take 1,000 mcg by mouth daily.",f,Oral,NaN,ordered,1000.0,mcg,NaN,NaN,NaN,NaN,NaN,NaN,epGSpBw4bXkctvVxMQKSRjlX-SWaJ5izz998FKEfDPGhb07mil2EFJELlvNcwUPjJbmSPqHhNf.Y8oHOdgIqMrxZGUKPjtqH4ns6HWG9soOU3,NaN,NaN,left_only
515,468343,2022-09-03,openepic_shared_sutter_health,eFHuAXuFjmhJR4KquDp6rEbZmMGDIac8B0.vYl8AVf5o3,active,order,Medication/eoV.dS6xAImSBX91MZRXtmKnLYHZRSQH6tnMFYVvkSBhfKBh5ChG-8H8ySXpAOC3T1QPu79RFY-.nAAQ4Ve411eO4DXY5A.2Ks4jhAEZqrZk3,sertraline 50mg Tab,2022-05-25T00:24:10Z,NaN,Erica L,Short course (acute) therapy,Take 50 mg by mouth daily,Take 50 mg by mouth daily,f,Oral,Take,ordered,50.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eoV.dS6xAImSBX91MZRXtmKnLYHZRSQH6tnMFYVvkSBhfKBh5ChG-8H8ySXpAOC3T1QPu79RFY-.nAAQ4Ve411eO4DXY5A.2Ks4jhAEZqrZk3,NaN,NaN,left_only
602,569629,2022-08-19,openepic_shared_sutter_health,eMH3jpK22KKbIXH.SXKepqQ09EYrB74sCOYKRGi6l0Mg3,active,order,Medication/elbEVBHbdCsnBC7SLF9eWrDi7luTQP3yRS.l6WJ-YB8nex0Ntd85FPeLWDe39lQTMB3k-gcghHKv.1IEOJi6R5HIYf3E1w-Kn9NYYdl7bsEE3,MAGNESIUM PO,2019-02-01T22:27:41Z,NaN,"Uchechukwu Mordi, PHARM STU",Short course (acute) therapy,Take 500 mg by mouth daily \r\n,Take 500 mg by mouth daily \r\n,f,Oral,Take,ordered,500.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,elbEVBHbdCsnBC7SLF9eWrDi7luTQP3yRS.l6WJ-YB8nex0Ntd85FPeLWDe39lQTMB3k-gcghHKv.1IEOJi6R5HIYf3E1w-Kn9NYYdl7bsEE3,NaN,NaN,left_only
603,569629,2022-08-19,openepic_shared_sutter_health,ewPFrYIl-S0C0uz1CiL6Tm-l

In [57]:
len(fhir_meds_ref.drop_duplicates('id'))

4255

In [55]:
len(fhir_meds)

16973

In [58]:
fhir_meds_ref

,resourcetype,id,text,text.1,coding_system,coding_code
0,Medication,ebsMyWHIq6lZJoDrjnnnF3G5RqiSqqEbm5vavJYlQfFj.EGEpR9MTEo4swV0YO2VSe37bB5uPCSufRNjDMNJ8WG0KugcBa6ly8-EsSvDXvvA3,amphetamine-dextroamphetamine 20 mg 24 hr capsule,"Capsule, Sust. Release 24 hr",http://www.nlm.nih.gov/research/umls/rxnorm,3288
1,Medication,eavp8p5K1s74DXuHcKaCud4056lOwgvNcAjnE1b3EAM9OnIOSNaeYRpu7Jo3yeT7hkAVLKj9B-BDa2Gt0CLqzqQ9-ywERSrU6gsTkBhjj39o3,tamsulosin 0.4 mg Cap,Capsule,http://www.nlm.nih.gov/research/umls/rxnorm,77492
2,Medication,eavp8p5K1s74DXuHcKaCud4056lOwgvNcAjnE1b3EAM9OnIOSNaeYRpu7Jo3yeT7hkAVLKj9B-BDa2Gt0CLqzqQ9-ywERSrU6gsTkBhjj39o3,tamsulosin 0.4 mg Cap,Capsule,http://www.nlm.nih.gov/research/umls/rxnorm,863671
3,Medication,eV7gbJQ8STW3qg6JzCwgEZzCgmpG0fFRFN1ZBweRs9k.M4zLsAs0PFGJ8tJeiShVEvnO5Z.XKTSJfUY9n856PYAq6B.U36T-BTdBOqCi1f783,TIMOPTIC OCUDOSE (PF) 0.5 % Dropper,Dropperette,http://www.nlm.nih.gov/research/umls/rxnorm,1922895
4,Medication,ekCagbHi8B-tgXvE-q9JPM8xX5hpjSdadTKKxX8RmcLfnYYWmWGGoeTTCmtwottzBMHeLX8g3vfkaUKxQ1AJcpEk.RpiLGbsV1rdygJXP9y43,famotidine 20 mg tablet,tablet,http://www.nlm.nih.gov/research/umls/rxnorm,196458
...,...,...,...,...,...,...
49850,Medication,e8huXTH9Ly7VldEZHsULTmVU8ewAm9Kcz2MTQYyVHYLyRwbZVKdGzmPtP7sYUP9xLDgKQt2uzyKgObNyQ5YcxxiJ6NcGWYN0PMaIenmWRtRw3,ferrous sulfate Tab tablet,Tablet,http://www.nlm.nih.gov/research/umls/rxnorm,216851
49851,Medication,e8huXTH9Ly7VldEZHsULTmVU8ewAm9Kcz2MTQYyVHYLyRwbZVKdGzmPtP7sYUP9xLDgKQt2uzyKgObNyQ5YcxxiJ6NcGWYN0PMaIenmWRtRw3,ferrous sulfate Tab tablet,Tablet,http://www.nlm.nih.gov/research/umls/rxnorm,217102
49852,Medication,e8huXTH9Ly7VldEZHsULTmVU8ewAm9Kcz2MTQYyVHYLyRwbZVKdGzmPtP7sYUP9xLDgKQt2uzyKgObNyQ5YcxxiJ6NcGWYN0PMaIenmWRtRw3,ferrous sulfate Tab tablet,Tablet,http://www.nlm.nih.gov/research/umls/rxnorm,217789
49853,Medication,e8huXTH9Ly7VldEZHsULTmVU8ewAm9Kcz2MTQYyVHYLyRwbZVKdGzmPtP7sYUP9xLDgKQt2uzyKgObNyQ5YcxxiJ6NcGWYN0PMaIenmWRtRw3,ferrous sulfate Tab tablet,Tablet,http://www.nlm.nih.gov/research/umls/rxnorm,217791
